In [1]:
%pip install -U odp-sdk --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.1.1 requires cryptography<41,>=38.0.0, but you have cryptography 42.0.8 which is incompatible.
fastapi 0.95.1 requires pydantic!=1.7,!=1.7.1,!=1.7.2,!=1.7.3,!=1.8,!=1.8.1,<2.0.0,>=1.6.2, but you have pydantic 2.7.3 which is incompatible.
cognite-sdk 2.46.1 requires shapely==1.*, but you have shapely 2.0.4 which is incompatible.
azure-identity 1.6.1 requires msal-extensions~=0.3.0, but you have msal-extensions 1.1.0 which is incompatible.
adlfs 2023.1.0 requires azure-storage-blob>=12.12.0, but you have azure-storage-blob 12.9.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import pandas as pd
import requests
import io

In [2]:
from odp_sdk.client import OdpClient
from odp_sdk.dto import ResourceDto
from odp_sdk.dto.file_dto import FileMetadataDto
from odp_sdk.dto.table_spec import TableSpec # Table Specification
from odp_sdk.exc import OdpResourceNotFoundError
from odp_sdk.exc import OdpResourceExistsError, OdpValidationError
from odp_sdk.dto.tabular_store import TableStage

In [3]:
client = OdpClient()

# Simple Tabular Data Ingestion onto ODP
In this example notebook, we run through the process of downloading some well formatted tabular data into the workspace, creating a new data collection, and then uploading it to Ocean Data Platform.

## Download Data
We will be using data from WWF's Ocean Futures Programme as the source. You can either upload it to the workspace (Option 1) or download it from the website (Option 2).

In [4]:
# Option 1: Use a local file (uncomment this section if needed)

"""
# Replace 'file.xlsx' with the path to your Excel file in the workspace
excel_file_path = 'Oceans_Futures_data_v111823_265c8e6c45.xlsx'

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(excel_file_path, header=6)  # Adjust the header index as needed if there is data above the table

# Now you can work with the DataFrame 'df'
df
"""

"\n# Replace 'file.xlsx' with the path to your Excel file in the workspace\nexcel_file_path = 'Oceans_Futures_data_v111823_265c8e6c45.xlsx'\n\n# Read the Excel file into a Pandas DataFrame\ndf = pd.read_excel(excel_file_path, header=6)  # Adjust the header index as needed if there is data above the table\n\n# Now you can work with the DataFrame 'df'\ndf\n"

In [5]:
# Option 2: Download directly from the WWF Ocean Futures website (uncomment this section if needed)
url = 'https://stor.oceansfutures.org/oceansfuture-storage/assets/Oceans_Futures_data_v111823_265c8e6c45.xlsx'
response = requests.get(url)
excel_content = response.content

# Wrap the content in a BytesIO object
excel_bytes_io = io.BytesIO(excel_content)

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(excel_bytes_io, header=6)  # Adjust the header index as needed if there is data above the table

# Now you can work with the DataFrame 'df'
df

,EEZ name,Uninhabited?,ISO,biomass_delta_2030,catch_trend,foreign_fishing_effort,seafood_dependence,historical_conflict_intensity,MPA_in_EEZ,contested_EEZ,historical_fisheries_conflicts,GDP_pc
0,Albania,0,ALB,17.31,-0.03,0.131942,0.04,0.000000e+00,1.228631,0,1,5030.5950
1,Algeria,0,DZA,3.30,-0.03,0.001806,0.05,1.627508e-06,0.059201,0,0,4238.6720
2,American Samoa,0,ASM,-6.00,-0.06,0.091612,0.26,0.000000e+00,8.696112,0,0,4330.1663
3,Andaman & Nicobar Isl. (India),0,IND,-1.02,0.01,0.000044,NaN,4.983078e-07,0.000000,0,0,NaN
4,Angola,0,AGO,2.38,-0.07,0.176665,0.31,4.840050e-07,1.865140,0,0,3536.8423
...,...,...,...,...,...,...,...,...,...,...,...,...
276,Viet Nam,0,VNM,-2.08,-0.04,0.035412,0.30,0.000000e+00,1.146355,0,14,3124.0996
277,Wake Isl. (USA),1,UMI,2.11,0.00,0.000015,NaN,0.000000e+00,99.481620,0,0,NaN
278,Wallis & Futuna Isl. (France),0,WLF,-5.10,-0.03,0.000173,0.31,0.000000e+00,0.000000,0,0,NaN
279,Yemen (Arabian Sea),0,YEM,0.00,0.01,0.000758,0.08,1.786539e-04,0.246836,0,0,972.3514


## Uploading tabular data to ODP
In this section we run through the process of writting tabluar data from start to finish which includes:
- Create a data collection
- Create a dataset
- Create the table schema
- Write the data to ODP
  
Some of these actions can be done on the front end, so you might be able to skip the first two steps

### Create a data collection (if needed)
Datasets sit within data collections on ODP so you will need to create one first. If you have already

In [ ]:
# need to create a data collection

In [8]:
my_collection = ResourceDto(
    kind="catalog.hubocean.io/dataCollection",
    version="v1alpha1",
    metadata={
        "name": "oceans-futures-fisheries-conflict-hotspot-database-KTCR19j",
        "display_name": "WWF Oceans Futures",
        "description": "Oceans Futures fisheries conflict hotspot database",
        "labels": {
            "hubocean.io/test": 'true'
        }
    },
    spec={
        "distribution": {
            "published_by": {
                "contact": "Ocean Smith <ocean@earth.org>",
                "organisation": "Ocean"
            },
            "published_date": "2019-06-19T06:00:00",
            "website": "https://ocean.earth",
            "license": {
                "name": "propriatary",
                "full_text": "This is a very strict legal text describing the data license for this data collection. The lawyer would be proud.",
                "href": "www.license.com"
            }
        },
        "tags": ["test", "hubocean"]
    }
)

# The catalog collection is created.
my_collection = client.catalog.create(my_collection)

### Create the dataset entry (if needed)
If you already created the dataset entry in the front end then you don't need to follow this step, but you can do it directly in the workspace.

In [9]:
my_dataset = ResourceDto(
    **{
        "kind": "catalog.hubocean.io/dataset",
        "version": "v1alpha3",
        "metadata": {
            "name": "oceans-futures-fisheries-conflict-hotspot-database-1317dw2", # computer friendly name for the dataset, add a random string at the end
            "display_name": "Ocean Futures Fisheries Conflict Hotspot Database", # human friendly name to display in the frontend
            "description": "The Oceans Futures platform provides actionable data to drive global priorities to stop fisheries conflict." # a short description of the dataset
        },
        "spec": {
            "storage_controller": "registry.hubocean.io/storageController/storage-tabular",
            "storage_class": "registry.hubocean.io/storageClass/tabular",
            "maintainer": {"contact": "Ocean Smith <ocean@earth.org>"}, # Your name and email address
            "data_collection": "catalog.hubocean.io/dataCollection/oceans-futures-fisheries-conflict-hotspot-database-KTCR19j", # The data collection you would like to place this dataset in (either created above or in the front end)

        },
    }
)

# The dataset is created in the catalog.
my_dataset = client.catalog.create(my_dataset)

### Create a metadata schema for the table
The metadata schema describes the type of data in each column and uses the schema available here: https://docs.hubocean.earth/sdk/table-spec/
You can do this manually or test using a code snippet that maps the 

#### Option 1: Automatic mapping
This option automatically writes the metadata schema using the dtypes fucntion in python. If it doesn't work you will need to try option 2.

In [11]:
def get_table_schema(df):
    dtype_mapping = {
        'bool': 'bool',
        'int64': 'int64',
        'float64': 'double',
        'object': 'string',
        'bytes': 'binary'
    }
    
    table_schema = {}
    for column, dtype in df.dtypes.items():
        # Convert dtype to string
        dtype_str = str(dtype)
        
        # Map the dtype to the schema type
        schema_type = dtype_mapping.get(dtype_str)
        
        # Check if the dtype is one of the predefined dtypes
        if schema_type:
            table_schema[column] = {"type": schema_type}
        else:
            # If dtype is not mapped, set it to string by default (or raise an error if preferred)
            table_schema[column] = {"type": "string"}
    
    return table_schema

table_schema = get_table_schema(df)

# Create a TableSpec
my_table_spec = TableSpec(table_schema=table_schema)
mt_table_spec = client.tabular.create_schema(resource_dto=my_dataset, table_spec=my_table_spec)
print(mt_table_spec)

table_schema={'EEZ name': {'type': 'string'}, 'contested_EEZ': {'type': 'int64'}, 'foreign_fishing_effort': {'type': 'double'}, 'historical_fisheries_conflicts': {'type': 'int64'}, 'ISO': {'type': 'string'}, 'seafood_dependence': {'type': 'double'}, 'GDP_pc': {'type': 'double'}, 'catch_trend': {'type': 'double'}, 'historical_conflict_intensity': {'type': 'double'}, 'Uninhabited?': {'type': 'int64'}, 'MPA_in_EEZ': {'type': 'double'}, 'biomass_delta_2030': {'type': 'double'}} partitioning=None


#### Option 2: Manual mapping
If option 1 doesn't work then you will need to write the table manually. This is how to do it for ther

In [ ]:
"""

table_schema = {
    "EEZ name": {"type": "string"},
    "Uninhabited?": {"type": "int64"},
    "ISO": {"type": "string"},
    "biomass_delta_2030": {"type": "double"},
    "catch_trend": {"type": "double"},
    "foreign_fishing_effort": {"type": "double"},
    "seafood_dependence": {"type": "double"},
    "historical_conflict_intensity": {"type": "double"},
    "MPA_in_EEZ": {"type": "double"},
    "contested_EEZ": {"type": "int64"},
    "historical_fisheries_conflicts": {"type": "int64"},
    "GDP_pc": {"type": "double"},
}

my_table_spec = TableSpec(table_schema=table_schema)

mt_table_spec = client.tabular.create_schema(resource_dto=my_dataset, table_spec=my_table_spec)
print(mt_table_spec)

"""

In [13]:
client.tabular.write_dataframe(resource_dto=my_dataset, data=df)
print("Dataset's data:", client.tabular.select_as_list(my_dataset))

Dataset's data: [{'EEZ name': 'Albania', 'contested_EEZ': 0, 'foreign_fishing_effort': 0.131942, 'historical_fisheries_conflicts': 1, 'ISO': 'ALB', 'seafood_dependence': 0.04, 'GDP_pc': 5030.594999999999, 'catch_trend': -0.03, 'historical_conflict_intensity': 0.0, 'Uninhabited?': 0, 'MPA_in_EEZ': 1.228631, 'biomass_delta_2030': 17.31}, {'EEZ name': 'Algeria', 'contested_EEZ': 0, 'foreign_fishing_effort': 0.001806, 'historical_fisheries_conflicts': 0, 'ISO': 'DZA', 'seafood_dependence': 0.05, 'GDP_pc': 4238.6720000000005, 'catch_trend': -0.03, 'historical_conflict_intensity': 1.6275080459221656e-06, 'Uninhabited?': 0, 'MPA_in_EEZ': 0.059201, 'biomass_delta_2030': 3.3}, {'EEZ name': 'American Samoa', 'contested_EEZ': 0, 'foreign_fishing_effort': 0.091612, 'historical_fisheries_conflicts': 0, 'ISO': 'ASM', 'seafood_dependence': 0.26, 'GDP_pc': 4330.166300000001, 'catch_trend': -0.06, 'historical_conflict_intensity': 0.0, 'Uninhabited?': 0, 'MPA_in_EEZ': 8.696112, 'biomass_delta_2030': -6.